# Лабораторная работа №4 "Проведение исследований со случайным лесом"

### Ход работы

Импортируем библиотеки перед работой

In [24]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import randint
from sklearn.exceptions import ConvergenceWarning
from sklearn.tree import DecisionTreeClassifier


##### Создание бейзлайна для модели классификации

Проведём те же манипуляции, что и ранне: выгрузим датасет и минимально его обработаем

In [10]:
c_base_df = pd.read_csv("../classification.csv").sample(frac=1, random_state=42).reset_index(drop=True)

c_base_df = c_base_df.drop(columns=['instance_id', 'obtained_date', 'track_name', 'artist_name'])

c_base_df.drop_duplicates()

c_base_df['tempo'] = pd.to_numeric(c_base_df['tempo'], errors='coerce')

le = LabelEncoder()
c_base_df['mode'] = le.fit_transform(c_base_df['mode'])
c_base_df['music_genre'] = le.fit_transform(c_base_df['music_genre'])
c_base_df['key'] = le.fit_transform(c_base_df['key'])

median_tempo = c_base_df['tempo'].median()
c_base_df['tempo'] = c_base_df['tempo'].fillna(median_tempo)

X_c_base = c_base_df.drop(columns=["music_genre"])
y_c_base = c_base_df["music_genre"]

X_c_base_train, X_c_base_test, y_c_base_train, y_c_base_test = train_test_split(
    X_c_base,
    y_c_base,
    test_size=0.2,
    random_state=42,
    stratify=y_c_base
)

Теперь обучим модель из sklearn

In [5]:
rf_model = RandomForestClassifier(
    n_estimators=100, 
    random_state=42,
    max_depth=3
)

rf_model.fit(X_c_base_train, y_c_base_train)

y_pred = rf_model.predict(X_c_base_test)

accuracy = accuracy_score(y_c_base_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.4723


##### Улучшение бейзлайна для модели классификации

In [6]:
print(classification_report(y_c_base_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.01      0.02      1000
           1       0.56      0.64      0.60      1000
           2       0.38      0.34      0.36      1000
           3       0.68      0.85      0.76      1000
           4       0.61      0.41      0.49      1000
           5       0.41      0.61      0.49      1000
           6       0.45      0.73      0.55      1000
           7       0.46      0.09      0.15      1000
           8       0.39      0.17      0.24      1000
           9       0.38      0.86      0.53      1000

    accuracy                           0.47     10000
   macro avg       0.49      0.47      0.42     10000
weighted avg       0.49      0.47      0.42     10000



Это лучше чем одно решающее дерево. Теперь преобразуем датасет как ранее

In [16]:
c_df = pd.read_csv("../classification.csv").sample(frac=1, random_state=42).reset_index(drop=True)

c_df = c_df.drop(columns=['instance_id', 'obtained_date', 'track_name', 'artist_name'])


sc = StandardScaler()
scaled = sc.fit_transform(c_df[['loudness', 'acousticness', 'energy']]) # ранее я не замечал, что loudness просто съедала остальные фичи 
pca = PCA(n_components=2)
c_df[['pc1', 'pc2']] = pca.fit_transform(scaled)
c_df = c_df.drop(columns=['loudness', 'acousticness', 'energy'])

c_df['tempo'] = pd.to_numeric(c_df['tempo'], errors='coerce')

le = LabelEncoder()
c_df['music_genre'] = le.fit_transform(c_df['music_genre'])
c_df['mode'] = le.fit_transform(c_df['mode'])

ohe = OneHotEncoder(sparse_output=False, drop='first')
encoded_key = ohe.fit_transform(c_df[['key']])
encoded_df_key = pd.DataFrame(encoded_key, columns=ohe.get_feature_names_out(['key']))
c_df = c_df.drop(columns=['key']).reset_index(drop=True)
c_df = pd.concat([c_df, encoded_df_key], axis=1)

c_df['duration_ms'] = c_df['duration_ms'].replace(-1, np.nan)

c_df['instrumental_flag'] = (c_df['instrumentalness'] > 0.05).astype(int)
c_df = c_df.drop(columns=['instrumentalness'])

c_df['undefined_tempo'] = c_df['tempo'].isna().astype(int)

median_tempo = c_df['tempo'].median()
c_df['tempo'] = c_df['tempo'].fillna(median_tempo)

median_duration = c_df['duration_ms'].median()
c_df['duration_ms'] = c_df['duration_ms'].fillna(median_duration)

float_features = [
    'popularity', 'danceability', 'duration_ms',
    'liveness', 'speechiness', 'tempo',
    'valence', 'pc1', 'pc2'
]

other_features = [
    'mode', 'instrumental_flag', 'undefined_tempo'
] + list(encoded_df_key.columns) 

X_c = c_df[float_features + other_features]
y_c = c_df['music_genre']

X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(
    X_c, y_c, test_size=0.2, stratify=y_c, random_state=42
)

Перейдём к обучению

In [ ]:
rf = RandomForestClassifier(
    random_state=42,
    n_jobs=1
)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_grid,
    n_iter=30,
    scoring='accuracy',
    cv=cv,
    n_jobs=1,  
    verbose=1,
    random_state=42
)

grid_search.fit(X_c_train, y_c_train)
best_model = grid_search.best_estimator_

print("Best params:", grid_search.best_params_)

y_pred = best_model.predict(X_c_test)

print("Score:", accuracy_score(y_c_test, y_pred))

print(classification_report(y_c_test, y_pred))

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best params: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 20}
Score: 0.543
              precision    recall  f1-score   support

           0       0.45      0.32      0.38      1000
           1       0.77      0.74      0.75      1000
           2       0.59      0.51      0.54      1000
           3       0.82      0.83      0.83      1000
           4       0.56      0.57      0.56      1000
           5       0.62      0.60      0.61      1000
           6       0.34      0.38      0.36      1000
           7       0.53      0.50      0.51      1000
           8       0.30      0.28      0.29      1000
           9       0.48      0.70      0.57      1000

    accuracy                           0.54     10000
   macro avg       0.55      0.54      0.54     10000
weighted avg       0.55      0.54      0.54     10000



##### Создание бейзлайна для модели регрессии

Сделаем всё то же, что и ранее

In [19]:
r_base_df = pd.read_csv("../regression.csv").sample(frac=1, random_state=42).reset_index(drop=True)

r_base_df['Date'] = pd.to_datetime(r_base_df['Date'], dayfirst=True)

r_base_df["Year"] = r_base_df["Date"].dt.year
r_base_df["Month"] = r_base_df["Date"].dt.month
r_base_df["Day"] = r_base_df["Date"].dt.day

r_base_df = r_base_df.drop(columns=['Date'])

per_store_count = r_base_df.groupby('Store').size().iloc[0]
k = max(1, int(np.round(0.8 * per_store_count))) 
store_counts = r_base_df['Store'].nunique()

train = r_base_df.iloc[: store_counts * k]
test = r_base_df.iloc[store_counts * k :]

X_r_base_train = train.drop(columns=['Weekly_Sales'])
X_r_base_test = test.drop(columns=['Weekly_Sales'])
y_r_base_train = train['Weekly_Sales']
y_r_base_test = test['Weekly_Sales']

И теперь обучим модель

In [15]:
rf_regressor = RandomForestRegressor(
    n_estimators=100, 
    random_state=42,
    max_depth=3
)

rf_regressor.fit(X_r_base_train, y_r_base_train)

y_pred = rf_regressor.predict(X_r_base_test)

mae = mean_absolute_error(y_r_base_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_r_base_test, y_pred))
r2 = r2_score(y_r_base_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

MAE: 329429.9998288926
RMSE: 432505.5663012246
R2: 0.42180745594044


##### Улучшение бейзлайна для модели регрессии

Сначала повторим техники из предыдущей ЛР

In [21]:
r_df = pd.read_csv("../regression.csv").sample(frac=1, random_state=42).reset_index(drop=True)

r_df['Date'] = pd.to_datetime(r_df['Date'], dayfirst=True)

r_df['Year'] = r_df['Date'].dt.year
r_df['Week'] = r_df['Date'].dt.isocalendar().week

r_df = r_df.drop(columns=['Date'])

cat_store = ['Store', 'Week']
other_feats = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Holiday_Flag']

per_store_count = r_df.groupby('Store').size().iloc[0]
k = max(1, int(np.round(0.8 * per_store_count))) 
store_counts = r_df['Store'].nunique()

train = r_df.iloc[: store_counts * k]
test = r_df.iloc[store_counts * k :]

X_r_train = train.drop(columns=['Weekly_Sales'])
X_r_test = test.drop(columns=['Weekly_Sales'])
y_r_train = train['Weekly_Sales']
y_r_test = test['Weekly_Sales']

warnings.filterwarnings("ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_store),
        ('passth', 'passthrough', other_feats)
    ],
    remainder='drop'
)

Перейдём к обучению

In [ ]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestRegressor(random_state=42))
])

param_dist = {
    'rf__n_estimators': [100, 200, 300],
    'rf__max_depth': [None, 5, 10, 15, 20],
    'rf__min_samples_split': [2, 5],
    'rf__min_samples_leaf': [1, 2],
    'rf__max_features': ['sqrt', 'log2']
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_dist,
    scoring='neg_mean_squared_error',
    cv=cv,
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_r_train, y_r_train)

print("Best params:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_test_pred  = best_model.predict(X_r_test)

mae = mean_absolute_error(y_r_test, y_test_pred)
rmse = np.sqrt(mean_squared_error(y_r_test, y_test_pred))
r2 = r2_score(y_r_test, y_test_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
Best params: {'rf__max_depth': None, 'rf__max_features': 'sqrt', 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 300}
MAE: 63864.84128812266
RMSE: 120223.71316758188
R2: 0.9553245064363924


##### Базовый класс имплементации

In [ ]:
class BaseRandomForest:
    def __init__(
        self,
        n_estimators=100,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        max_depth=None,
    ):
        self.n_estimators = n_estimators
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.max_depth = max_depth
        self.trees_ = []

    def _create_tree(self):
        """Создание одного дерева (Classifier или Regressor)."""
        raise NotImplementedError

    def _aggregate_predictions(self, predictions):
        """Агрегация предсказаний деревьев."""
        raise NotImplementedError

    def _bootstrap_sample(self, X, y):
        """
        Bootstrap-сэмплирование.
        """
        n_samples = X.shape[0]
        indices = np.random.randint(0, n_samples, n_samples)

        # pandas
        if hasattr(X, "iloc"):
            return X.iloc[indices], y.iloc[indices]

        # numpy
        return X[indices], y[indices]

    def _fit_single_tree(self, X, y):
        """
        Обучение одного дерева.
        """
        # Bootstrap
        X_sample, y_sample = self._bootstrap_sample(X, y)

        # Создание дерева
        tree = self._create_tree()

        # Обучение дерева
        tree.fit(X_sample, y_sample)
        return tree


    def fit(self, X, y):
        """
        Обучение случайного леса (последовательно).
        """
        self.trees_ = []

        for _ in range(self.n_estimators):
            tree = self._fit_single_tree(X, y)
            self.trees_.append(tree)

        return self

    def predict(self, X):
        """
        Предсказание леса.
        """
        predictions = np.asarray([tree.predict(X) for tree in self.trees_])
        return self._aggregate_predictions(predictions)


##### Классификатор

In [ ]:
class MyRandomForestClassifier(BaseRandomForest):

    def _create_tree(self):
        """
        Создание одного дерева классификации.
        Все гиперпараметры прокидываются из леса.
        Здесь и далее будут использоваться деревья из sklearn для экономии времени
        и более честного сравнения самого алгоритма
        """
        return DecisionTreeClassifier(
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            max_features=self.max_features,
            max_depth=self.max_depth,
        )

    def _aggregate_predictions(self, predictions):
        """
        Агрегация предсказаний деревьев.
        Используется голосование большинством.
        """
        n_samples = predictions.shape[1]
        y_pred = np.empty(n_samples, dtype=predictions.dtype)

        # Голосование по каждому объекту
        for i in range(n_samples):
            values, counts = np.unique(predictions[:, i], return_counts=True)
            y_pred[i] = values[np.argmax(counts)]

        return y_pred


Обучим на данных бейзлайна

In [23]:
rf_model = MyRandomForestClassifier(
    n_estimators=100,
    max_depth=3
)

rf_model.fit(X_c_base_train, y_c_base_train)

y_pred = rf_model.predict(X_c_base_test)

accuracy = accuracy_score(y_c_base_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_c_base_test, y_pred))

Accuracy: 0.4179
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1000
           1       0.71      0.14      0.23      1000
           2       0.42      0.26      0.32      1000
           3       0.71      0.83      0.76      1000
           4       0.32      0.55      0.40      1000
           5       0.36      0.71      0.48      1000
           6       0.38      0.89      0.53      1000
           7       0.38      0.03      0.05      1000
           8       0.00      0.00      0.00      1000
           9       0.40      0.78      0.53      1000

    accuracy                           0.42     10000
   macro avg       0.37      0.42      0.33     10000
weighted avg       0.37      0.42      0.33     10000



Обучим на данных улучшенной модели. Сразу будем использовать параметры, полученные при подборе гиперпараметров

In [17]:
rf_model = MyRandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2
)

rf_model.fit(X_c_train, y_c_train)

y_pred = rf_model.predict(X_c_test)

print("Score:", accuracy_score(y_c_test, y_pred))

print(classification_report(y_c_test, y_pred))

Score: 0.5437
              precision    recall  f1-score   support

           0       0.40      0.38      0.39      1000
           1       0.75      0.75      0.75      1000
           2       0.57      0.53      0.55      1000
           3       0.83      0.83      0.83      1000
           4       0.57      0.56      0.56      1000
           5       0.63      0.60      0.62      1000
           6       0.36      0.47      0.40      1000
           7       0.57      0.46      0.51      1000
           8       0.28      0.20      0.23      1000
           9       0.48      0.66      0.56      1000

    accuracy                           0.54     10000
   macro avg       0.54      0.54      0.54     10000
weighted avg       0.54      0.54      0.54     10000



Имеем скоры:

**Бейзлайн sklearn** - 0.4723

**Улучшенная модель sklearn** - 0.543

**Мой бейзлайн** - 0.4179

**Мой улучшенный бейзлайн** - 0.5437

Моя модель не отличается по скору от модели sklearn на улучшенных версиях

##### Регрессор

In [18]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor


class MyRandomForestRegressor(BaseRandomForest):
    """
    Случайный лес для регрессии.

    Наследуется от BaseRandomForest и реализует:
    - создание регрессионного дерева
    - агрегацию предсказаний усреднением
    """

    def _create_tree(self):
        """
        Создание одного дерева регрессии.
        """
        return DecisionTreeRegressor(
            min_samples_split=self.min_samples_split,
            min_samples_leaf=self.min_samples_leaf,
            max_features=self.max_features,
            max_depth=self.max_depth,
        )

    def _aggregate_predictions(self, predictions):
        """
        Агрегация предсказаний деревьев.

        predictions shape:
        (n_estimators, n_samples)

        Для регрессии используется усреднение.
        """
        return predictions.mean(axis=0)


Обучим на данных бейзлайна

In [20]:
rf_regressor = MyRandomForestRegressor(
    n_estimators=100, 
    max_depth=3
)

rf_regressor.fit(X_r_base_train, y_r_base_train)

y_pred = rf_regressor.predict(X_r_base_test)

mae = mean_absolute_error(y_r_base_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_r_base_test, y_pred))
r2 = r2_score(y_r_base_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

MAE: 390951.74828000314
RMSE: 471079.7424955274
R2: 0.31407292980488344


Обучим на данных улучшенной модели. Сразу будем использовать параметры, полученные при подборе гиперпараметров

In [22]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', MyRandomForestRegressor(min_samples_leaf=1, min_samples_split=2, n_estimators=300))
])

pipeline.fit(X_r_train, y_r_train)

y_test_pred  = pipeline.predict(X_r_test)

mae = mean_absolute_error(y_r_test, y_test_pred)
rmse = np.sqrt(mean_squared_error(y_r_test, y_test_pred))
r2 = r2_score(y_r_test, y_test_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

MAE: 64193.45753407412
RMSE: 120375.05403558185
R2: 0.9552119581977049


Получили

| Метрика | Бейзлайн случайного леса | Бейзлайн имплементации | Улучшенный случайный лес | Улучшенная имплементация |
|-|-|-|-|-|
| MAE | 329430 | 390952 | 63865 | 64193 |
| RMSE | 432506 | 471080 | 120224 | 120375 |
| R2 | 0.421 | 0.314 | 0.955 | 0.955 |

Практически одинаковые метрики на улучшенных версиях